# Raw Data Processing

## DBLP paper ciation data loading

In [255]:
file_path = '/Users/zongchang/Desktop/实验数据集/DBLP-Paper'

In [256]:
with open(file_path + '/DBLPOnlyCitationOct19.txt', 'r') as f:
    paper_list = []
    paper = {}
    paper_raw = []
    for line in f.readlines():
        if line == '\n':
            ref_index = []
            for item in paper_raw:
                if item.startswith('#*'):
                    paper['title'] = item.replace('#*', '')
                elif item.startswith('#@'):
                    paper['author'] = item.replace('#@', '')
                elif item.startswith('#t'):
                    paper['year'] = item.replace('#t', '')
                elif item.startswith('#c'):
                    paper['venue'] = item.replace('#c', '')
                elif item.startswith('#index'):
                    paper['index'] = item.replace('#index', '')
                elif item.startswith('#%'):
                    ref_index.append(item.replace('#%', ''))
                elif item.startswith('#!'):
                    paper['abstract'] = item.replace('#!', '')
                else:
                    continue
            paper['ref_index'] = ref_index
            paper_list.append(paper)
            paper = {}
            paper_raw = []
        else:
            paper_raw.append(line.strip())

In [257]:
# filter paper with low quality: absence of properties, earlier papers, small number of references
paper_list_filter = []
for paper in paper_list:
    if ('abstract' not in paper) or ('title' not in paper) or ('author' not in paper) or (int(paper['year']) < 2000) or len(paper['ref_index'])<10:
        continue
    else:
        paper_list_filter.append(paper)

In [258]:
len(paper_list_filter)

66501

## DBLP paper citation count (indicator_input.pkl)

In [259]:
# index yearly citation dict for papers at year 2000
index_cite_count = {}
for paper in paper_list_filter:
    if paper['year'] == '2000':
        index_cite_count[paper['index']] = {}

# count citation
for paper in paper_list_filter:
    for ref in paper['ref_index']:
        if ref in index_cite_count:
            if paper['year'] not in index_cite_count[ref]:
                index_cite_count[ref][paper['year']] = 1
            else:
                index_cite_count[ref][paper['year']] += 1

        

In [260]:
# keep papers have citations
index_cite_count_nonzero = {}
for k,v in index_cite_count.items():
    if v:
        index_cite_count_nonzero[k] = v

In [261]:
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
citation_count_list = [] # index, year, citation_count
for k,v in index_cite_count_nonzero.items():
    for year in years:
        if year not in v:
            tmp = [k, year + '-12-31', 0]
        else:
            tmp = [k, year + '-12-31', v[year]]
        citation_count_list.append(tmp)

In [262]:
citation_count_list

[['5652', '2000-12-31', 0],
 ['5652', '2001-12-31', 0],
 ['5652', '2002-12-31', 0],
 ['5652', '2003-12-31', 1],
 ['5652', '2004-12-31', 4],
 ['5652', '2005-12-31', 0],
 ['5652', '2006-12-31', 1],
 ['5652', '2007-12-31', 2],
 ['5652', '2008-12-31', 1],
 ['5652', '2009-12-31', 4],
 ['5652', '2010-12-31', 0],
 ['5693', '2000-12-31', 0],
 ['5693', '2001-12-31', 0],
 ['5693', '2002-12-31', 2],
 ['5693', '2003-12-31', 0],
 ['5693', '2004-12-31', 1],
 ['5693', '2005-12-31', 0],
 ['5693', '2006-12-31', 1],
 ['5693', '2007-12-31', 2],
 ['5693', '2008-12-31', 0],
 ['5693', '2009-12-31', 1],
 ['5693', '2010-12-31', 0],
 ['6516', '2000-12-31', 1],
 ['6516', '2001-12-31', 0],
 ['6516', '2002-12-31', 0],
 ['6516', '2003-12-31', 0],
 ['6516', '2004-12-31', 0],
 ['6516', '2005-12-31', 0],
 ['6516', '2006-12-31', 0],
 ['6516', '2007-12-31', 1],
 ['6516', '2008-12-31', 0],
 ['6516', '2009-12-31', 0],
 ['6516', '2010-12-31', 0],
 ['9722', '2000-12-31', 0],
 ['9722', '2001-12-31', 0],
 ['9722', '2002-12-3

In [263]:
import pickle
with open('./data/dblp_paper/paper_1749_citation.pkl', 'wb') as f:
    pickle.dump(citation_count_list, f)

In [314]:
idx_2_paper_index = {}
idx = 0
for paper_index in list(index_cite_count_nonzero.keys()):
    idx_2_paper_index[idx] = paper_index
    idx += 1

In [315]:
indicator_citation = []
for idx in range(len(list(idx_2_paper_index.keys()))):
    tmp = []
    for indicator in citation_count_list:
        if indicator[0] == idx_2_paper_index[idx]:
            tmp.append([[indicator[2]], indicator[1]])
    
    indicator_citation.append(tmp)

In [316]:
indicator_citation

[[[[0], '2000-12-31'],
  [[0], '2001-12-31'],
  [[0], '2002-12-31'],
  [[1], '2003-12-31'],
  [[4], '2004-12-31'],
  [[0], '2005-12-31'],
  [[1], '2006-12-31'],
  [[2], '2007-12-31'],
  [[1], '2008-12-31'],
  [[4], '2009-12-31'],
  [[0], '2010-12-31']],
 [[[0], '2000-12-31'],
  [[0], '2001-12-31'],
  [[2], '2002-12-31'],
  [[0], '2003-12-31'],
  [[1], '2004-12-31'],
  [[0], '2005-12-31'],
  [[1], '2006-12-31'],
  [[2], '2007-12-31'],
  [[0], '2008-12-31'],
  [[1], '2009-12-31'],
  [[0], '2010-12-31']],
 [[[1], '2000-12-31'],
  [[0], '2001-12-31'],
  [[0], '2002-12-31'],
  [[0], '2003-12-31'],
  [[0], '2004-12-31'],
  [[0], '2005-12-31'],
  [[0], '2006-12-31'],
  [[1], '2007-12-31'],
  [[0], '2008-12-31'],
  [[0], '2009-12-31'],
  [[0], '2010-12-31']],
 [[[0], '2000-12-31'],
  [[0], '2001-12-31'],
  [[2], '2002-12-31'],
  [[0], '2003-12-31'],
  [[0], '2004-12-31'],
  [[0], '2005-12-31'],
  [[1], '2006-12-31'],
  [[1], '2007-12-31'],
  [[0], '2008-12-31'],
  [[1], '2009-12-31'],
  [[0], 

In [267]:
import pickle
with open('./data/dblp_paper/indicator_input.pkl', 'wb') as f:
    pickle.dump(indicator_citation, f)

## DBLP paper graph (graph_input.pkl)

In [317]:
# extract author and citation relations from high quality papers
paper_author = {}
paper_citation = {}
author_list = []
paper_list = []
years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']

for item in paper_list_filter:
    author_list.extend(item['author'].split(','))
    paper_list.append(item['index'])
    paper_author[item['index']] = {'author': item['author'].split(','), 'year': item['year']}
    paper_citation[item['index']] = {'paper': item['ref_index'], 'year': item['year']}
    

In [318]:
author_list = list(set(author_list))
paper_list = list(set(paper_list))

In [319]:
len(list(paper_author.keys()))

66501

In [320]:
len(list(paper_citation.keys()))

66501

In [321]:
# continue add paper index
exist_paper_index = list(idx_2_paper_index.values())
idx = max(list(idx_2_paper_index.keys())) + 1
for paper_index in paper_list:
    if paper_index in exist_paper_index:
        continue
    else:
        idx_2_paper_index[idx] = paper_index
        idx += 1

In [322]:
max(list(idx_2_paper_index.keys()))

66500

In [323]:
idx_2_author_name = {}
idx = 70000
for author_name in author_list:
    idx_2_author_name[idx] = author_name
    idx += 1

In [324]:
author_name_2_idx = {}
for k,v in idx_2_author_name.items():
    author_name_2_idx[v] = k

paper_index_2_idx = {}
for k,v in idx_2_paper_index.items():
    paper_index_2_idx[v] = k

In [325]:
idx_2_author_name

{70000: 'Kouichi Sakurai',
 70001: 'Sathish Gopalakrishnan',
 70002: 'Hemant Tagare',
 70003: 'Jonathan de Halleux',
 70004: 'Karina Hauser',
 70005: 'Zhigang Peng',
 70006: 'Edwin K. P. Chong',
 70007: 'Michael Dittenbach',
 70008: 'Yuan-Tse Yu',
 70009: 'Paul Thompson',
 70010: 'Thierry Mora',
 70011: 'Natalie Lee-San Pang',
 70012: 'Traci A. Carte',
 70013: 'Z. M. Ma',
 70014: 'Junyu Dong',
 70015: 'Emmanuel Juin',
 70016: 'Deng Cai',
 70017: "Andrea D'Ambrogio",
 70018: 'Janne Riihijärvi',
 70019: 'James A. Storer',
 70020: 'Jagoba Arias',
 70021: 'Riko Jacob',
 70022: 'Peter R. Wilson',
 70023: 'Ravi Janardan',
 70024: 'Darrell C. Anderson',
 70025: 'Yingwei Cui',
 70026: 'Alan Halverson',
 70027: 'Matthew Suderman',
 70028: 'Reetuparna Das',
 70029: 'Christopher Schwarz',
 70030: 'Ferdi A. Smit',
 70031: 'Su-Ping Chen',
 70032: 'Daniela Giorgetti',
 70033: 'Zhongnan Zhang',
 70034: 'Vishal J. Mehta',
 70035: 'Rami Beidas',
 70036: 'Robert C. Goldstein',
 70037: 'Farshad Fotouhi',

In [326]:
len(list(idx_2_author_name.keys()))

74395

In [327]:
idx_2_paper_index

{0: '5652',
 1: '5693',
 2: '6516',
 3: '9722',
 4: '13378',
 5: '17568',
 6: '17575',
 7: '17759',
 8: '19853',
 9: '20105',
 10: '20181',
 11: '20383',
 12: '20494',
 13: '20534',
 14: '20634',
 15: '20952',
 16: '21087',
 17: '21116',
 18: '23291',
 19: '23292',
 20: '23293',
 21: '39252',
 22: '39260',
 23: '39305',
 24: '39518',
 25: '39789',
 26: '39861',
 27: '41420',
 28: '41428',
 29: '41440',
 30: '41447',
 31: '41473',
 32: '41523',
 33: '41546',
 34: '41588',
 35: '41600',
 36: '41604',
 37: '41671',
 38: '41672',
 39: '44834',
 40: '50066',
 41: '53584',
 42: '53600',
 43: '53610',
 44: '53625',
 45: '53665',
 46: '53701',
 47: '53713',
 48: '53732',
 49: '53755',
 50: '53758',
 51: '53761',
 52: '53791',
 53: '53831',
 54: '53835',
 55: '60916',
 56: '60939',
 57: '60955',
 58: '61021',
 59: '61048',
 60: '61053',
 61: '61078',
 62: '61125',
 63: '62742',
 64: '71315',
 65: '71927',
 66: '77045',
 67: '77100',
 68: '77108',
 69: '77195',
 70: '77277',
 71: '77314',
 72: '

In [328]:
paper_author_idx = {}
for k,v in paper_author.items():
    paper_author_idx[paper_index_2_idx[k]] = {'author': [author_name_2_idx[item] for item in v['author']], 'year': v['year']}

In [329]:
paper_author_idx

{23333: {'author': [82210, 126930], 'year': '2003'},
 54982: {'author': [88346, 123957], 'year': '2003'},
 51116: {'author': [103107, 123957, 123010], 'year': '2003'},
 26669: {'author': [91157], 'year': '2003'},
 2882: {'author': [135240, 112147, 98020], 'year': '2003'},
 26654: {'author': [123348, 82552, 123995], 'year': '2003'},
 23472: {'author': [77339, 127058, 139042], 'year': '2003'},
 3309: {'author': [119626, 118034, 104362, 80829], 'year': '2003'},
 31232: {'author': [87459, 73641, 122377, 101879, 82983], 'year': '2003'},
 42141: {'author': [82210, 126930], 'year': '2002'},
 50805: {'author': [106374, 94159], 'year': '2005'},
 50136: {'author': [91947], 'year': '2005'},
 11522: {'author': [84390, 80487, 140799, 122593], 'year': '2003'},
 35816: {'author': [118425, 125463, 114507], 'year': '2003'},
 62122: {'author': [116840, 132181], 'year': '2003'},
 55456: {'author': [85925, 78041], 'year': '2003'},
 24617: {'author': [113519], 'year': '2003'},
 9508: {'author': [113519, 81

In [330]:
paper_citation_idx = {}
for k,v in paper_citation.items():
    tmp = []
    for item in v['paper']:
        if item in paper_index_2_idx:
            tmp.append(item)
    paper_citation_idx[paper_index_2_idx[k]] = {'paper': [paper_index_2_idx[item] for item in tmp], 'year': v['year']}

In [331]:
paper_citation_idx

{23333: {'paper': [829, 1500, 17973], 'year': '2003'},
 54982: {'paper': [1000, 829, 17973], 'year': '2003'},
 51116: {'paper': [1300], 'year': '2003'},
 26669: {'paper': [], 'year': '2003'},
 2882: {'paper': [], 'year': '2003'},
 26654: {'paper': [52990, 64141], 'year': '2003'},
 23472: {'paper': [], 'year': '2003'},
 3309: {'paper': [1357, 758, 18613, 1036], 'year': '2003'},
 31232: {'paper': [19949, 708], 'year': '2003'},
 42141: {'paper': [1500], 'year': '2002'},
 50805: {'paper': [37465, 650], 'year': '2005'},
 50136: {'paper': [27679, 42152, 642, 1360, 1363], 'year': '2005'},
 11522: {'paper': [1360, 52990], 'year': '2003'},
 35816: {'paper': [1363], 'year': '2003'},
 62122: {'paper': [582], 'year': '2003'},
 55456: {'paper': [], 'year': '2003'},
 24617: {'paper': [], 'year': '2003'},
 9508: {'paper': [1363], 'year': '2003'},
 55401: {'paper': [], 'year': '2003'},
 46181: {'paper': [1363], 'year': '2003'},
 55864: {'paper': [60897], 'year': '2003'},
 44204: {'paper': [26329], 'ye

In [352]:
# idx to paper title
idx_2_paper_title = {}

for item in paper_list_filter:
    if item['index'] in paper_index_2_idx and item['index'] not in idx_2_paper_title:
        idx_2_paper_title[paper_index_2_idx[item['index']]] = item['title']

In [360]:
idx_2_paper_title[1]

'PaperButtons: Expanding a Tangible User Interface.'

In [355]:
import networkx as nx

years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010']
graph_list = []
for year in years:
    G_author = nx.Graph()
    G_citation = nx.Graph()
    G_author.add_nodes_from(list(idx_2_paper_index.keys()))
    G_author.add_nodes_from(list(idx_2_author_name.keys()))
    G_citation.add_nodes_from(list(idx_2_paper_index.keys()))
    G_citation.add_nodes_from(list(idx_2_author_name.keys()))

    for k,v in paper_author_idx.items():
        if v['year'] == year:
            for author in v['author']:
                G_author.add_edge(k, author)

    for k,v in paper_citation_idx.items():
        if v['year'] == year:
            for paper in v['paper']:
                G_citation.add_edge(k, paper)
    graph_list.append([G_author, G_citation, idx_2_paper_title, idx_2_author_name])


In [356]:

for i in range(len(graph_list)):
    print(years[i] + '  Author edges added this year: ' + str(nx.number_of_edges(graph_list[i][0])) + '  Citation edges added this year: ' + str(nx.number_of_edges(graph_list[i][1])))


2000  Author edges added this year: 5591  Citation edges added this year: 700
2001  Author edges added this year: 7289  Citation edges added this year: 2409
2002  Author edges added this year: 11369  Citation edges added this year: 5863
2003  Author edges added this year: 14973  Citation edges added this year: 11031
2004  Author edges added this year: 15720  Citation edges added this year: 15673
2005  Author edges added this year: 18381  Citation edges added this year: 22307
2006  Author edges added this year: 21897  Citation edges added this year: 29890
2007  Author edges added this year: 31517  Citation edges added this year: 46847
2008  Author edges added this year: 31171  Citation edges added this year: 50386
2009  Author edges added this year: 32528  Citation edges added this year: 56900
2010  Author edges added this year: 8247  Citation edges added this year: 15060


In [357]:
for i in range(len(graph_list)):
    print(years[i] + '  Snapshot nodes this year: ' + str(nx.number_of_nodes(graph_list[i][0])))

2000  Snapshot nodes this year: 140896
2001  Snapshot nodes this year: 140896
2002  Snapshot nodes this year: 140896
2003  Snapshot nodes this year: 140896
2004  Snapshot nodes this year: 140896
2005  Snapshot nodes this year: 140896
2006  Snapshot nodes this year: 140896
2007  Snapshot nodes this year: 140896
2008  Snapshot nodes this year: 140896
2009  Snapshot nodes this year: 140896
2010  Snapshot nodes this year: 140896


In [358]:
with open('./data/dblp_paper/graph_input.pkl', 'wb') as f:
    pickle.dump(graph_list, f)

In [336]:
paper_num = {}
for year in years:
    tmp = []
    for k,v in paper_citation_idx.items():
        if v['year'] == year:
            tmp.append(k)
    paper_num[year] = len(list(set(tmp)))

In [337]:
for k,v in paper_num.items():
    print(k + '  Papers published this year: ' + str(v))

2000  Papers published this year: 2102
2001  Papers published this year: 2703
2002  Papers published this year: 4155
2003  Papers published this year: 5333
2004  Papers published this year: 5390
2005  Papers published this year: 6208
2006  Papers published this year: 7299
2007  Papers published this year: 10377
2008  Papers published this year: 10081
2009  Papers published this year: 10290
2010  Papers published this year: 2563


## DBLP paper text content (doc_input.pkl)

In [338]:
paper_index_list = []
for item in citation_count_list:
    if item[0] not in paper_index_list:
        paper_index_list.append(item[0])

In [339]:
len(paper_index_list)

1749

In [340]:
doc_input = []
for paper_index in paper_index_list:
    tmp_item = []
    for paper in paper_list_filter:
        if paper['index'] == paper_index:
            tmp_item.append([[paper['title'], paper['abstract']], paper['year']+'-12-31'])
    doc_input.append(tmp_item)

In [341]:
doc_input

[[[['Video Artifacts for Design: Bridging the Gap Between Abstraction and Detail.',
    'Video artifacts help bridge the gap between abstraction and detail in the design process. This paper describes how our use and re-use of video artifacts affected the re-design of a graphical editor for building, simulating, and analyzing Coloured Petri Nets. The two primary goals of the project were to create design abstractions that integrate recent advances in graphical interaction techniques and to explicitly support specific patterns of use of Petri nets in real-world settings.Using a participatory design process, we organized a series of video-based design activities that helped us manage the tension between finding useful design abstractions and specifying the details of the user interface. Video artifacts resulting from one activity became the basis for the next, facilitating communication among members of the multi-disciplinary design team. The video artifacts provided an efficient way of c

In [342]:
import pickle
with open('./data/dblp_paper/doc_input.pkl', 'wb') as f:
    pickle.dump(doc_input, f)

## DBLP paper citation trend label (trend_label.pkl)

In [345]:
# trend label: 0 for up, 1 for down, 2 for no change
trend_label = []
for paper_citation in indicator_citation:
    prev_citation = None
    tmp = []
    for year_citation in paper_citation:
        current_citation = year_citation[0][0]
        if prev_citation is not None:
            if current_citation > prev_citation:
                tmp.append([0, year_citation[1]])
            elif current_citation < prev_citation:
                tmp.append([1, year_citation[1]])
            else:
                tmp.append([2, year_citation[1]])
            prev_citation = current_citation
        else:
            prev_citation = current_citation
            continue
    trend_label.append(tmp)

In [346]:
trend_label

[[[2, '2001-12-31'],
  [2, '2002-12-31'],
  [0, '2003-12-31'],
  [0, '2004-12-31'],
  [1, '2005-12-31'],
  [0, '2006-12-31'],
  [0, '2007-12-31'],
  [1, '2008-12-31'],
  [0, '2009-12-31'],
  [1, '2010-12-31']],
 [[2, '2001-12-31'],
  [0, '2002-12-31'],
  [1, '2003-12-31'],
  [0, '2004-12-31'],
  [1, '2005-12-31'],
  [0, '2006-12-31'],
  [0, '2007-12-31'],
  [1, '2008-12-31'],
  [0, '2009-12-31'],
  [1, '2010-12-31']],
 [[1, '2001-12-31'],
  [2, '2002-12-31'],
  [2, '2003-12-31'],
  [2, '2004-12-31'],
  [2, '2005-12-31'],
  [2, '2006-12-31'],
  [0, '2007-12-31'],
  [1, '2008-12-31'],
  [2, '2009-12-31'],
  [2, '2010-12-31']],
 [[2, '2001-12-31'],
  [0, '2002-12-31'],
  [1, '2003-12-31'],
  [2, '2004-12-31'],
  [2, '2005-12-31'],
  [0, '2006-12-31'],
  [2, '2007-12-31'],
  [1, '2008-12-31'],
  [0, '2009-12-31'],
  [1, '2010-12-31']],
 [[2, '2001-12-31'],
  [2, '2002-12-31'],
  [0, '2003-12-31'],
  [0, '2004-12-31'],
  [0, '2005-12-31'],
  [1, '2006-12-31'],
  [0, '2007-12-31'],
  [1, '20

In [347]:
with open('./data/dblp_paper/trend_label.pkl', 'wb') as f:
    pickle.dump(trend_label, f)

## AI patent ciation data loading

In [1]:
patent_data_path = '/Users/zongchang/Desktop/实验数据集/Patent-AI'
header_map = {'标题':'title', '摘要':'abstract', '申请人':'applicants', '公开（公告）号':'public_id',
              '公开（公告）日':'public_date', '专利类型':'patent_type', '公开国别':'public_country', '标题（翻译）':'title_trans',
              '摘要（翻译）':'abstract_trans','发明人':'inventors', '引证专利':'references','CPC':'cpc'}

In [2]:
# keep patents published in US, with references and later than 2013-01-01
import pandas as pd
import datetime

file_num = 137
patents = []

for i in range(file_num):
    df = pd.read_excel(patent_data_path+'/'+str(i+1)+'.xls')
    df = df[list(header_map.keys())]
    df.rename(columns=header_map, inplace=True)
    df = df.fillna('')
    tmp = df.to_dict(orient='records')
    for item in tmp:
        patent = {}
        patent['pub_id'] = item['public_id']
        # process citation relations
        if item['references']:
            cite_list = item['references'].split('; ')
            patent['reference'] = cite_list
        # process cpc relations
        if item['cpc']:
            cpc_list = item['cpc'].split('; ')
            patent['cpc'] = cpc_list
        # process public dates
        if item['public_date'] and item['public_date'] >= datetime.datetime.strptime('2013-01-01', '%Y-%m-%d'):
            public_date = item['public_date'].strftime("%Y-%m-%d")
            patent['pub_date'] = public_date
        # process applicants
        if item['applicants']:
            applicant_list = item['applicants'].split('; ')
            patent['applicant'] = applicant_list
        # process inventors
        if item['inventors']:
            inventor_list = item['inventors'].split('; ')
            patent['inventor'] = inventor_list
        # process public country
        if item['public_country'] and item['public_country'] == 'US':
            patent['country'] = item['public_country']
        # process patent type
        if item['patent_type'] and item['patent_type'] == '发明申请':
            patent['type'] = item['patent_type']
        # process title
        if item['title_trans'] and item['public_country']=='CN':
            patent['title'] = item['title_trans']
        if item['title'] and item['public_country']!='CN':
            patent['title'] = item['title']
        # process abstract
        if item['abstract_trans'] and item['public_country']=='CN':
            patent['abstract'] = item['abstract_trans']
        if item['abstract'] and item['public_country']!='CN':
            patent['abstract'] = item['abstract']
        patents.append(patent)
    print('processed file' + str(i+1))

processed file1
processed file2
processed file3
processed file4
processed file5
processed file6
processed file7
processed file8
processed file9
processed file10
processed file11
processed file12
processed file13
processed file14
processed file15
processed file16
processed file17
processed file18
processed file19
processed file20
processed file21
processed file22
processed file23
processed file24
processed file25
processed file26
processed file27
processed file28
processed file29
processed file30
processed file31
processed file32
processed file33
processed file34
processed file35
processed file36
processed file37
processed file38
processed file39
processed file40
processed file41
processed file42
processed file43
processed file44
processed file45
processed file46
processed file47
processed file48
processed file49
processed file50
processed file51
processed file52
processed file53
processed file54
processed file55
processed file56
processed file57
processed file58
processed file59
proces

In [3]:
keys = ['pub_id', 'reference', 'cpc', 'pub_date', 'applicant', 'inventor', 'country', 'type', 'title', 'abstract']
patent_list_filter = []
for patent in patents:
    if len(list(patent.keys())) == len(keys):
        patent_list_filter.append(patent)

In [5]:
len(patent_list_filter)

81540

## AI patent citation count (indicator_input.pkl)

In [62]:
# index yearly citation dict for patents at year 201
index_cite_count = {}
for patent in patent_list_filter:
    if patent['pub_date'][:4] == '2013':
        index_cite_count[patent['pub_id']] = {}

# count citation
for patent in patent_list_filter:
    for ref in patent['reference']:
        if ref in index_cite_count:
            if patent['pub_date'][:4] not in index_cite_count[ref]:
                index_cite_count[ref][patent['pub_date'][:4]] = 1
            else:
                index_cite_count[ref][patent['pub_date'][:4]] += 1

In [63]:
# keep patents have citations
index_cite_count_nonzero = {}
for k,v in index_cite_count.items():
    if v:
        index_cite_count_nonzero[k] = v

In [70]:
years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
citation_count_list = [] # index, year, citation_count
for k,v in index_cite_count_nonzero.items():
    for year in years:
        if year not in v:
            tmp = [k, year + '-12-31', 0]
        else:
            tmp = [k, year + '-12-31', v[year]]
        citation_count_list.append(tmp)

In [71]:
len(list(index_cite_count_nonzero.keys()))

3537

In [75]:
citation_count_list

[['US20130031971A1', '2013-12-31', 1],
 ['US20130031971A1', '2014-12-31', 0],
 ['US20130031971A1', '2015-12-31', 0],
 ['US20130031971A1', '2016-12-31', 0],
 ['US20130031971A1', '2017-12-31', 0],
 ['US20130031971A1', '2018-12-31', 0],
 ['US20130031971A1', '2019-12-31', 0],
 ['US20130032421A1', '2013-12-31', 0],
 ['US20130032421A1', '2014-12-31', 0],
 ['US20130032421A1', '2015-12-31', 1],
 ['US20130032421A1', '2016-12-31', 0],
 ['US20130032421A1', '2017-12-31', 0],
 ['US20130032421A1', '2018-12-31', 1],
 ['US20130032421A1', '2019-12-31', 1],
 ['US20130033484A1', '2013-12-31', 0],
 ['US20130033484A1', '2014-12-31', 0],
 ['US20130033484A1', '2015-12-31', 2],
 ['US20130033484A1', '2016-12-31', 0],
 ['US20130033484A1', '2017-12-31', 0],
 ['US20130033484A1', '2018-12-31', 0],
 ['US20130033484A1', '2019-12-31', 0],
 ['US20130033485A1', '2013-12-31', 3],
 ['US20130033485A1', '2014-12-31', 1],
 ['US20130033485A1', '2015-12-31', 2],
 ['US20130033485A1', '2016-12-31', 1],
 ['US20130033485A1', '201

In [11]:
import pickle
with open('./data/ai_patent/patent_3537_citation.pkl', 'wb') as f:
    pickle.dump(citation_count_list, f)

In [74]:
idx_2_patent_id = {}
idx = 0
for patent_id in list(index_cite_count_nonzero.keys()):
    idx_2_patent_id[idx] = patent_id
    idx += 1

In [76]:
indicator_citation = []
for idx in range(len(list(idx_2_patent_id.keys()))):
    tmp = []
    for indicator in citation_count_list:
        if indicator[0] == idx_2_patent_id[idx]:
            tmp.append([[indicator[2]], indicator[1]])
    
    indicator_citation.append(tmp)

In [77]:
indicator_citation

[[[[1], '2013-12-31'],
  [[0], '2014-12-31'],
  [[0], '2015-12-31'],
  [[0], '2016-12-31'],
  [[0], '2017-12-31'],
  [[0], '2018-12-31'],
  [[0], '2019-12-31']],
 [[[0], '2013-12-31'],
  [[0], '2014-12-31'],
  [[1], '2015-12-31'],
  [[0], '2016-12-31'],
  [[0], '2017-12-31'],
  [[1], '2018-12-31'],
  [[1], '2019-12-31']],
 [[[0], '2013-12-31'],
  [[0], '2014-12-31'],
  [[2], '2015-12-31'],
  [[0], '2016-12-31'],
  [[0], '2017-12-31'],
  [[0], '2018-12-31'],
  [[0], '2019-12-31']],
 [[[3], '2013-12-31'],
  [[1], '2014-12-31'],
  [[2], '2015-12-31'],
  [[1], '2016-12-31'],
  [[3], '2017-12-31'],
  [[2], '2018-12-31'],
  [[3], '2019-12-31']],
 [[[0], '2013-12-31'],
  [[1], '2014-12-31'],
  [[2], '2015-12-31'],
  [[0], '2016-12-31'],
  [[1], '2017-12-31'],
  [[2], '2018-12-31'],
  [[1], '2019-12-31']],
 [[[0], '2013-12-31'],
  [[0], '2014-12-31'],
  [[0], '2015-12-31'],
  [[0], '2016-12-31'],
  [[1], '2017-12-31'],
  [[0], '2018-12-31'],
  [[0], '2019-12-31']],
 [[[0], '2013-12-31'],
  [[1

In [78]:
import pickle
with open('./data/ai_patent/indicator_input.pkl', 'wb') as f:
    pickle.dump(indicator_citation, f)

## AI patent graph (graph_input.pkl)

In [81]:
# extract cpc and citation relations from high quality patents
patent_cpc = {}
patent_citation = {}
cpc_list = []
patent_list = []
years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019']

for item in patent_list_filter:
    cpc_list.extend(item['cpc'])
    patent_list.append(item['pub_id'])
    patent_cpc[item['pub_id']] = {'cpc': item['cpc'], 'year': item['pub_date'][:4]}
    patent_citation[item['pub_id']] = {'patent': item['reference'], 'year': item['pub_date'][:4]}

In [84]:
cpc_list = list(set(cpc_list))
patent_list = list(set(patent_list))

In [85]:
len(list(patent_cpc.keys()))

81530

In [86]:
len(list(patent_citation.keys()))

81530

In [87]:
# continue add patent id
exist_patent_id = list(idx_2_patent_id.values())
idx = max(list(idx_2_patent_id.keys())) + 1
for patent_id in patent_list:
    if patent_id in exist_patent_id:
        continue
    else:
        idx_2_patent_id[idx] = patent_id
        idx += 1

In [88]:
max(list(idx_2_patent_id.keys()))

81529

In [89]:
idx_2_cpc_name = {}
idx = 90000
for cpc_name in cpc_list:
    idx_2_cpc_name[idx] = cpc_name
    idx += 1

In [90]:
cpc_name_2_idx = {}
for k,v in idx_2_cpc_name.items():
    cpc_name_2_idx[v] = k

patent_id_2_idx = {}
for k,v in idx_2_patent_id.items():
    patent_id_2_idx[v] = k

In [91]:
idx_2_cpc_name

{90000: 'G08B29/188',
 90001: 'G05D1/0202',
 90002: 'A61G7/1003',
 90003: 'B05B12/12',
 90004: 'G01L1/04',
 90005: 'A61N2/06',
 90006: 'B62D1/28',
 90007: 'H04N21/4122',
 90008: 'B62B5/026',
 90009: 'G01S13/91',
 90010: 'Y02E10/52',
 90011: 'G01J1/4228',
 90012: 'G01N2800/365',
 90013: 'A61B5/0071',
 90014: 'C09J11/00',
 90015: 'F28F21/04',
 90016: 'H04N21/4408',
 90017: 'G01F5/00',
 90018: 'A61G11/00',
 90019: 'G02B21/241',
 90020: 'G01C21/3605',
 90021: 'A61H11/00',
 90022: 'G11B20/00449',
 90023: 'A45C2011/002',
 90024: 'A63B23/032',
 90025: 'B25J15/0483',
 90026: 'B60N2/5891',
 90027: 'G11B2220/90',
 90028: 'B60K2370/788',
 90029: 'B60Q2300/3321',
 90030: 'A61B1/0669',
 90031: 'G06F16/2255',
 90032: 'G01N2201/06',
 90033: 'G09G3/3677',
 90034: 'G06Q30/0215',
 90035: 'G01N2015/1062',
 90036: 'G09B23/32',
 90037: 'G02B21/0036',
 90038: 'G06K15/4025',
 90039: 'H04M2250/04',
 90040: 'F02D2200/606',
 90041: 'G01S19/43',
 90042: 'B60B27/0015',
 90043: 'G05B2219/37559',
 90044: 'B60S11/00

In [92]:
len(list(idx_2_cpc_name.keys()))

37613

In [93]:
idx_2_patent_id

{0: 'US20130031971A1',
 1: 'US20130032421A1',
 2: 'US20130033484A1',
 3: 'US20130033485A1',
 4: 'US20130033522A1',
 5: 'US20130033608A1',
 6: 'US20130033614A1',
 7: 'US20130033756A1',
 8: 'US20130034264A1',
 9: 'US20130034265A1',
 10: 'US20130034266A1',
 11: 'US20130034267A1',
 12: 'US20130034268A1',
 13: 'US20130034272A1',
 14: 'US20130034274A1',
 15: 'US20130034277A1',
 16: 'US20130034279A1',
 17: 'US20130034282A1',
 18: 'US20130034283A1',
 19: 'US20130034291A1',
 20: 'US20130034293A1',
 21: 'US20130034296A1',
 22: 'US20130034298A1',
 23: 'US20130034302A1',
 24: 'US20130035563A1',
 25: 'US20130035575A1',
 26: 'US20130035596A1',
 27: 'US20130035690A1',
 28: 'US20130035742A1',
 29: 'US20130035790A1',
 30: 'US20130035805A1',
 31: 'US20130035821A1',
 32: 'US20130035837A1',
 33: 'US20130035871A1',
 34: 'US20130035901A1',
 35: 'US20130036134A1',
 36: 'US20130036471A1',
 37: 'US20130026220A1',
 38: 'US20130026945A1',
 39: 'US20130027184A1',
 40: 'US20130027196A1',
 41: 'US20130027293A1',
 4

In [95]:
patent_cpc_idx = {}
for k,v in patent_cpc.items():
    patent_cpc_idx[patent_id_2_idx[k]] = {'cpc': [cpc_name_2_idx[item] for item in v['cpc']], 'year': v['year']}

In [96]:
patent_cpc_idx

{46266: {'cpc': [90817,
   91229,
   113440,
   123490,
   115611,
   100035,
   108839,
   112197,
   96302,
   96830,
   126109,
   100358,
   123681,
   120901,
   114662,
   119729,
   97043,
   102438,
   115520,
   106687],
  'year': '2020'},
 16868: {'cpc': [102585,
   109637,
   90802,
   117501,
   127608,
   123776,
   101749,
   123213,
   122432,
   112468,
   111898,
   125549,
   118926,
   106169,
   125733,
   99830,
   120746,
   119000,
   119396,
   104994,
   91902,
   110226,
   95643,
   102381,
   113132,
   102125,
   120841,
   122355],
  'year': '2020'},
 26030: {'cpc': [90799,
   102585,
   122155,
   123206,
   99988,
   111627,
   95785,
   116033,
   127074,
   111637,
   92819,
   100828,
   111013,
   121423,
   93278,
   106164,
   123807],
  'year': '2020'},
 49904: {'cpc': [127074,
   92733,
   109402,
   91404,
   115846,
   101483,
   106635,
   105139,
   93321],
  'year': '2020'},
 7070: {'cpc': [119499,
   97101,
   102767,
   108404,
   123367,


In [97]:
patent_citation_idx = {}
for k,v in patent_citation.items():
    tmp = []
    for item in v['patent']:
        if item in patent_id_2_idx:
            tmp.append(item)
    patent_citation_idx[patent_id_2_idx[k]] = {'patent': [patent_id_2_idx[item] for item in tmp], 'year': v['year']}

In [98]:
patent_citation_idx

{46266: {'patent': [80738, 15280, 29082], 'year': '2020'},
 16868: {'patent': [], 'year': '2020'},
 26030: {'patent': [64773, 24372], 'year': '2020'},
 49904: {'patent': [], 'year': '2020'},
 7070: {'patent': [], 'year': '2020'},
 46515: {'patent': [], 'year': '2020'},
 62993: {'patent': [], 'year': '2020'},
 36822: {'patent': [], 'year': '2020'},
 17635: {'patent': [], 'year': '2020'},
 44237: {'patent': [], 'year': '2020'},
 41130: {'patent': [34638, 66690], 'year': '2020'},
 29176: {'patent': [78163], 'year': '2020'},
 37094: {'patent': [68663], 'year': '2020'},
 28118: {'patent': [68663], 'year': '2020'},
 3546: {'patent': [17910], 'year': '2020'},
 16912: {'patent': [56035, 78488], 'year': '2020'},
 12673: {'patent': [], 'year': '2020'},
 70115: {'patent': [], 'year': '2020'},
 15204: {'patent': [51290], 'year': '2020'},
 73165: {'patent': [5001], 'year': '2020'},
 62771: {'patent': [], 'year': '2020'},
 22243: {'patent': [], 'year': '2020'},
 68967: {'patent': [66922, 32909], 'ye

In [99]:
# idx to paper title
idx_2_patent_title = {}

for item in patent_list_filter:
    if item['pub_id'] in patent_id_2_idx and item['pub_id'] not in idx_2_patent_title:
        idx_2_patent_title[patent_id_2_idx[item['pub_id']]] = item['title']

In [100]:
idx_2_patent_title[1]

'DRIVING ASSISTANCE APPARATUS FOR ASSISTANCE WITH DRIVING ALONG NARROW ROADWAYS'

In [101]:
import networkx as nx

years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
graph_list = []
for year in years:
    G_cpc = nx.Graph()
    G_citation = nx.Graph()
    G_cpc.add_nodes_from(list(idx_2_patent_id.keys()))
    G_cpc.add_nodes_from(list(idx_2_cpc_name.keys()))
    G_citation.add_nodes_from(list(idx_2_patent_id.keys()))
    G_citation.add_nodes_from(list(idx_2_cpc_name.keys()))

    for k,v in patent_cpc_idx.items():
        if v['year'] == year:
            for cpc in v['cpc']:
                G_cpc.add_edge(k, cpc)

    for k,v in patent_citation_idx.items():
        if v['year'] == year:
            for patent in v['patent']:
                G_citation.add_edge(k, patent)
    graph_list.append([G_cpc, G_citation, idx_2_patent_title, idx_2_cpc_name])

In [103]:
for i in range(len(graph_list)):
    print(years[i] + '  CPC edges added this year: ' + str(nx.number_of_edges(graph_list[i][0])) + '  Citation edges added this year: ' + str(nx.number_of_edges(graph_list[i][1])))

2013  CPC edges added this year: 60213  Citation edges added this year: 1111
2014  CPC edges added this year: 91644  Citation edges added this year: 3247
2015  CPC edges added this year: 116606  Citation edges added this year: 7159
2016  CPC edges added this year: 144066  Citation edges added this year: 10024
2017  CPC edges added this year: 159411  Citation edges added this year: 14236
2018  CPC edges added this year: 167080  Citation edges added this year: 16997
2019  CPC edges added this year: 172831  Citation edges added this year: 18672


In [104]:
for i in range(len(graph_list)):
    print(years[i] + '  Snapshot nodes this year: ' + str(nx.number_of_nodes(graph_list[i][0])))

2013  Snapshot nodes this year: 119143
2014  Snapshot nodes this year: 119143
2015  Snapshot nodes this year: 119143
2016  Snapshot nodes this year: 119143
2017  Snapshot nodes this year: 119143
2018  Snapshot nodes this year: 119143
2019  Snapshot nodes this year: 119143


In [105]:
with open('./data/ai_patent/graph_input.pkl', 'wb') as f:
    pickle.dump(graph_list, f)

In [106]:
patent_num = {}
for year in years:
    tmp = []
    for k,v in patent_citation_idx.items():
        if v['year'] == year:
            tmp.append(k)
    patent_num[year] = len(list(set(tmp)))

In [107]:
for k,v in patent_num.items():
    print(k + '  Patents published this year: ' + str(v))

2013  Patents published this year: 6410
2014  Patents published this year: 8591
2015  Patents published this year: 10807
2016  Patents published this year: 11455
2017  Patents published this year: 12406
2018  Patents published this year: 12840
2019  Patents published this year: 13323


## AI patent text content (doc_input.pkl)

In [108]:
patent_id_list = []
for item in citation_count_list:
    if item[0] not in patent_id_list:
        patent_id_list.append(item[0])

In [109]:
len(patent_id_list)

3537

In [110]:
doc_input = []
for patent_id in patent_id_list:
    tmp_item = []
    for patent in patent_list_filter:
        if patent['pub_id'] == patent_id:
            tmp_item.append([[patent['title'], patent['abstract']], patent['pub_date'][:4]+'-12-31'])
    doc_input.append(tmp_item)

In [111]:
doc_input

[[[['METHODS FOR MONITORING FLUIDS WITHIN OR PRODUCED FROM A SUBTERRANEAN FORMATION DURING FRACTURING OPERATIONS USING OPTICOANALYTICAL DEVICES',
    'In or near real-time monitoring of fluids can take place using an opticoanalytical device that is configured for monitoring the fluid. Fluids can be monitored prior to or during their introduction into a subterranean formation using the opticoanalytical devices. Produced fluids from a subterranean formation can be monitored in a like manner. The methods can comprise providing a fracturing fluid comprising a base fluid and at least one fracturing fluid component;  introducing the fracturing fluid into a subterranean formation at a pressure sufficient to create or enhance at least one fracture therein,  thereby performing a fracturing operation in the subterranean formation;  and monitoring a characteristic of the fracturing fluid or a formation fluid using at least a first opticoanalytical device within the subterranean formation,  during

In [112]:
import pickle
with open('./data/ai_patent/doc_input.pkl', 'wb') as f:
    pickle.dump(doc_input, f)

## AI patent citation trend label (trend_label.pkl)

In [113]:
# trend label: 0 for up, 1 for down, 2 for no change
trend_label = []
for patent_citation in indicator_citation:
    prev_citation = None
    tmp = []
    for year_citation in patent_citation:
        current_citation = year_citation[0][0]
        if prev_citation is not None:
            if current_citation > prev_citation:
                tmp.append([0, year_citation[1]])
            elif current_citation < prev_citation:
                tmp.append([1, year_citation[1]])
            else:
                tmp.append([2, year_citation[1]])
            prev_citation = current_citation
        else:
            prev_citation = current_citation
            continue
    trend_label.append(tmp)

In [114]:
trend_label

[[[1, '2014-12-31'],
  [2, '2015-12-31'],
  [2, '2016-12-31'],
  [2, '2017-12-31'],
  [2, '2018-12-31'],
  [2, '2019-12-31']],
 [[2, '2014-12-31'],
  [0, '2015-12-31'],
  [1, '2016-12-31'],
  [2, '2017-12-31'],
  [0, '2018-12-31'],
  [2, '2019-12-31']],
 [[2, '2014-12-31'],
  [0, '2015-12-31'],
  [1, '2016-12-31'],
  [2, '2017-12-31'],
  [2, '2018-12-31'],
  [2, '2019-12-31']],
 [[1, '2014-12-31'],
  [0, '2015-12-31'],
  [1, '2016-12-31'],
  [0, '2017-12-31'],
  [1, '2018-12-31'],
  [0, '2019-12-31']],
 [[0, '2014-12-31'],
  [0, '2015-12-31'],
  [1, '2016-12-31'],
  [0, '2017-12-31'],
  [0, '2018-12-31'],
  [1, '2019-12-31']],
 [[2, '2014-12-31'],
  [2, '2015-12-31'],
  [2, '2016-12-31'],
  [0, '2017-12-31'],
  [1, '2018-12-31'],
  [2, '2019-12-31']],
 [[0, '2014-12-31'],
  [1, '2015-12-31'],
  [2, '2016-12-31'],
  [2, '2017-12-31'],
  [2, '2018-12-31'],
  [2, '2019-12-31']],
 [[1, '2014-12-31'],
  [0, '2015-12-31'],
  [0, '2016-12-31'],
  [1, '2017-12-31'],
  [1, '2018-12-31'],
  [1, 

In [115]:
with open('./data/ai_patent/trend_label.pkl', 'wb') as f:
    pickle.dump(trend_label, f)